In [21]:
import os
import time
import torch
import zipfile
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models as models

from torch.cuda import amp
from google.colab import drive
from torchsummary import summary
from PIL import Image, ImageFile
from torchvision import transforms
from torchvision.io import read_image
from tqdm.notebook import tqdm, trange
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score, accuracy_score

ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/drive')

zip_file_path = '/content/drive/MyDrive/Colab Notebooks/KULIAH/PRAKTIKUM/annotations.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

Mounted at /content/drive


In [3]:
class ROP_Dataset(Dataset):
    def __init__(self, txt_file, image_dir, transform=None):
        self.image_list = pd.read_csv(txt_file, header=None)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_list.iloc[idx, 0])
        image = Image.open(img_path)
        img_tag = self.image_list.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        sample = {"image": image, "tag": img_tag}

        return image, img_tag

In [4]:
MEAN = torch.tensor((0.485, 0.456, 0.406))
STD  = torch.tensor((0.229, 0.224, 0.225))

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(15),
    # transforms.Normalize((0.5,), (0.5,))
])

In [7]:
# Create Dataset Training
dataset_train = ROP_Dataset(txt_file='./annotations/train.txt', image_dir='./annotations/images', transform=transform)
# Create Dataset Testing
dataset_test = ROP_Dataset(txt_file='./annotations/test.txt', image_dir='./annotations/images', transform=transform)
# Create Dataset Validation
dataset_valid = ROP_Dataset(txt_file='./annotations/valid.txt', image_dir='./annotations/images', transform=transform)

# DataLoader Training
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
# DataLoader Testing
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)
# DataLoader Validation
dataloader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=True)

print(dataloader_train)
print(dataloader_test)
print(dataloader_valid)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [14]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        # self.resnet = models.resnet18(weights=None)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

In [15]:
# Define model, criterion, and optimizer
num_classes = 2 # Healthy & Not Healthy
model = ResNetModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 86.9MB/s]


In [27]:
def train_model(learningRate, num_epochs):
  # Define Adam optimizer with the current learning rate
  optimizer = optim.Adam(model.parameters(), lr=learningRate)

  # Training loop
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      all_labels = []
      all_predictions = []
      for inputs, labels in dataloader_train:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * inputs.size(0)

          _, predicted = torch.max(outputs, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

      # Calculate average training loss per epoch
      epoch_loss = running_loss / len(dataset_train)
      precision = precision_score(all_labels, all_predictions, average='weighted')
      recall = recall_score(all_labels, all_predictions, average='weighted')
      accuracy = accuracy_score(all_labels, all_predictions)
      # print(f"LR : {learningRate} - Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")

  # Evaluation loop
  model.eval()
  all_labels = []
  all_predictions = []
  with torch.no_grad():
      for inputs, labels in dataloader_valid:
          inputs, labels = inputs.to(device), labels.to(device)

          outputs = model(inputs)

          _, predicted = torch.max(outputs, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

  precision = precision_score(all_labels, all_predictions, average='weighted')
  recall = recall_score(all_labels, all_predictions, average='weighted')
  accuracy = accuracy_score(all_labels, all_predictions)
  print(f"Learning Rate: {learningRate}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")


In [25]:
# Define learning rates
learning_rates = [0.001, 0.005, 0.01, 0.02, 0.03, 0.05, 0.08, 0.1, 0.15]

In [29]:
num_epochs = 50

for lr in learning_rates:
  train_model(lr, num_epochs)

Learning Rate: 0.001, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
Learning Rate: 0.005, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
Learning Rate: 0.01, Recall: 0.9714, Precision: 0.9714, Accuracy: 0.9714
Learning Rate: 0.02, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
Learning Rate: 0.03, Recall: 0.9571, Precision: 0.9610, Accuracy: 0.9571
Learning Rate: 0.05, Recall: 0.9571, Precision: 0.9610, Accuracy: 0.9571
Learning Rate: 0.08, Recall: 0.9000, Precision: 0.9378, Accuracy: 0.9000
Learning Rate: 0.1, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
Learning Rate: 0.15, Recall: 0.9571, Precision: 0.9591, Accuracy: 0.9571


In [ ]:
# Learning Rate: 0.001, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
# Learning Rate: 0.005, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
# Learning Rate: 0.01, Recall: 0.9714, Precision: 0.9714, Accuracy: 0.9714
# Learning Rate: 0.02, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
# Learning Rate: 0.03, Recall: 0.9571, Precision: 0.9610, Accuracy: 0.9571
# Learning Rate: 0.05, Recall: 0.9571, Precision: 0.9610, Accuracy: 0.9571
# Learning Rate: 0.08, Recall: 0.9000, Precision: 0.9378, Accuracy: 0.9000
# Learning Rate: 0.1, Recall: 0.9857, Precision: 0.9859, Accuracy: 0.9857
# Learning Rate: 0.15, Recall: 0.9571, Precision: 0.9591, Accuracy: 0.9571